In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report ,accuracy_score

# GPU 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Transform
transform = transforms.Compose([
    transforms.Resize((150, 150)),  
    transforms.ToTensor(),          
    transforms.Normalize((0.5,), (0.5,))
])

# Setting up the data
train_data = datasets.ImageFolder(root='data/train', transform=transform)
test_data = datasets.ImageFolder(root='data/test', transform=transform)

# Data loader
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64)

# Model 
class BrainTumorCNN(nn.Module):
    def __init__(self):
        super(BrainTumorCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)  
        self.pool = nn.MaxPool2d(2, 2)  
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(32 * 37 * 37, 128)  
        self.fc2 = nn.Linear(128, 4)  

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))  
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 32 * 37 * 37)  
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = BrainTumorCNN().to(device)

# Calculating the loss and setting up the acc
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training
num_epochs =25
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()           
        outputs = model(inputs)         
        loss = criterion(outputs, labels)  
        loss.backward()                
        optimizer.step()               

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}")

# Getting the acc
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.numpy())

print(classification_report(all_labels, all_preds, target_names=train_data.classes))
accuracy = accuracy_score(all_labels, all_preds)
print(f"Accuracy: {accuracy*100:.2f}%")

Epoch 1/25, Loss: 1.7449
Epoch 2/25, Loss: 0.8565
Epoch 3/25, Loss: 0.5166
Epoch 4/25, Loss: 0.4039
Epoch 5/25, Loss: 0.4175
Epoch 6/25, Loss: 0.3070
Epoch 7/25, Loss: 0.3917
Epoch 8/25, Loss: 0.2649
Epoch 9/25, Loss: 0.1817
Epoch 10/25, Loss: 0.1389
Epoch 11/25, Loss: 0.1208
Epoch 12/25, Loss: 0.1146
Epoch 13/25, Loss: 0.0761
Epoch 14/25, Loss: 0.0984
Epoch 15/25, Loss: 0.1063
Epoch 16/25, Loss: 0.0469
Epoch 17/25, Loss: 0.0377
Epoch 18/25, Loss: 0.0263
Epoch 19/25, Loss: 0.0260
Epoch 20/25, Loss: 0.0219
Epoch 21/25, Loss: 0.0132
Epoch 22/25, Loss: 0.0118
Epoch 23/25, Loss: 0.0071
Epoch 24/25, Loss: 0.0070
Epoch 25/25, Loss: 0.0043
              precision    recall  f1-score   support

          no       1.00      0.85      0.92        20
         yes       0.91      1.00      0.95        31

    accuracy                           0.94        51
   macro avg       0.96      0.93      0.94        51
weighted avg       0.95      0.94      0.94        51

Accuracy: 94.12%
